### Search Engine With Tools and Agents

In [1]:
## In built tools
## Arxiv--Website with research papers
## Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun #this interacts with the tool itself
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper #to interact with the tools we need a wrapper


In [ ]:
#using the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)


arxiv


In [5]:
#Combining these two tools
#combine them in the form of list
tools=[wiki,arxiv]


In [8]:
## Custom tools[RAG Tools]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [10]:
#I will load a website page and create that into a vectorstore and use it like my tool
loader=WebBaseLoader("https://docs.langchain.com/langsmith/home")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026F1580BDC0>, search_kwargs={})

In [17]:
#using the retriever we can make queries and get information, but in order to use it as a tool we have to convert the retriever into a tool
from langchain_core.tools import Tool

retriever_tool = Tool(
    name="langsmith_search",
    description="Search for information about LangSmith documentation",
    func=lambda query: retriever.invoke(query)
)


retriever_tool.name

'langsmith_search'

In [18]:
tools=[wiki,arxiv,retriever_tool]

In [19]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\HP\\anaconda3\\envs\\genaiprojects\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search for information about LangSmith documentation', func=<function <lambda> at 0x0000026F395EB6D0>)]

In [23]:
#Run all this tools with Agents and LLM Models
from langchain_groq import ChatGroq
import streamlit as st
#Tools, LLM-->AgentExecutor
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.3-70b-versatile")

ImportError: cannot import name 'hub' from 'langchainhub' (c:\Users\HP\anaconda3\envs\genaiprojects\lib\site-packages\langchainhub\__init__.py)